In [1]:
#importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#loading dataset
bankaccount_closures_df=pd.read_csv('/content/Bank_Predictions.csv')

#determining features and target
X = bankaccount_closures_df.iloc[:, 3:13].values #ignoring number, customer_id and last_name columns as not useful for prediction
y = bankaccount_closures_df.iloc[:, 13].values #loading acc_closed binary feature value as target

# Encoding categorical data of Location and Gender features
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

# Avoiding the dummy variable trap
X = X[:, 1:]


# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)




In [2]:
#building ANN


# import keras library and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

# Initializing the ANN
classifier = Sequential()

# You might use the following parameters: activation: 'relu' and/or 'sigmoid', optimization function is 'adam', loss function is 'binary_crossentropy', number of epochs is 100, samples per epoch are 10)
# Adding the input layer and the first hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=11))

# Adding second hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Adding output layer
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix : \n",cm)


Epoch 1/100
800/800 [==============================] - 2s 2ms/step - loss: 0.5028 - accuracy: 0.7969
Epoch 2/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4165 - accuracy: 0.8217
Epoch 3/100
800/800 [==============================] - 1s 2ms/step - loss: 0.4025 - accuracy: 0.8284
Epoch 4/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3919 - accuracy: 0.8305
Epoch 5/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3787 - accuracy: 0.8338
Epoch 7/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3742 - accuracy: 0.8425
Epoch 8/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3700 - accuracy: 0.8462
Epoch 9/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3673 - accuracy: 0.8476
Epoch 10/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3655 - accuracy: 0.8516
Epoch 11/100
800/800 [==============================] - 1s 2ms/step - loss: 0.3631 - accur

If the predicted probability is above a certain threshold (usually 0.5), you can classify the account as being at risk of closure.

 Confusion matrix provides a summary of the predictions made by a classification model on a dataset. The confusion matrix presents the counts of true positive (TP), true negative (TN), false positive (FP), and false negative (FN) predictions.


\begin{array}{cc|c}
    & \text{Predicted Negative} & \text{Predicted Positive} \\
\hline
\text{Actual Negative} & TN & FP \\
\text{Actual Positive} & FN & TP \\
\end{array}



In [3]:
#evaluating few performance metrics

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)

# Compute precision
precision = precision_score(y_test, y_pred)

# Compute recall
recall = recall_score(y_test, y_pred)

# Compute F1-score
f1 = f1_score(y_test, y_pred)

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Extract TN, FP, FN, TP from confusion matrix
TN, FP, FN, TP = cm.ravel()

# Compute specificity (True Negative Rate)
specificity = TN / (TN + FP)

# Compute false positive rate (FPR)
fpr = FP / (FP + TN)

# Compute false negative rate (FNR)
fnr = FN / (FN + TP)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Specificity:", specificity)
print("False Positive Rate (FPR):", fpr)
print("False Negative Rate (FNR):", fnr)


Accuracy: 0.856
Precision: 0.6869009584664537
Recall: 0.5308641975308642
F1-score: 0.5988857938718662
Specificity: 0.9385579937304075
False Positive Rate (FPR): 0.061442006269592474
False Negative Rate (FNR): 0.4691358024691358
